In [2]:
import pandas as pd
import numpy as np
import math

# Understanding Decision Trees

In [3]:
go_out_df = pd.read_csv('../../datasets/go-out2.csv')
go_out_df

,outlook,temp,humidity,windy,play
0,overcast,hot,high,False,yes
1,overcast,cool,normal,True,yes
2,overcast,mild,high,True,yes
3,overcast,hot,normal,False,yes
4,rainy,mild,high,False,yes
5,rainy,cool,normal,False,yes
6,rainy,cool,normal,True,no
7,rainy,mild,normal,False,yes
8,rainy,mild,high,True,no
9,sunny,hot,high,False,no


## ID3

### Steps
1. Calculate Entropy for each attribute
2. Calculated the avg sum of entropy for that feature
3. Calculate the Info Gain
3. Pick the attribute with highest gain
4. Repeat 1, 2, 3, 4 until a generalized tree has been created

## 1. Calculate Entropy

![Entropy](../images/entropy.png)

![Entropy](../images/entropy_2.png)

### Example: Calculate Entropy for Weather


## 2. Calculate the tnformation Gain		
The information gain is based on the decrease in entropy after a dataset is split on an attribute. Constructing a decision tree is all about finding attribute that returns the highest information gain (i.e., the most homogeneous branches).

![Information Gain](../images/info_gain.png)





In [4]:
go_out_df.play.value_counts()

yes    9
no     5
Name: play, dtype: int64

In [5]:
X = go_out_df[['outlook','temp','humidity','windy']]
y = go_out_df.play

In [6]:
def count_classes(label_serie):
    return label_serie.value_counts().to_dict()
def entropy(y):
    """Calculate the Gini Impurity for a list of rows."""
    counts = count_classes(y)
    entropy = 0
    # for each label calculate the entropy
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(y))
        entropy -= prob_of_lbl * math.log2(prob_of_lbl)
    return entropy

In [7]:
# Calculate the initial entropy (all dataset) 
entropy_root = entropy(y)
entropy_root

0.9402859586706309

In [8]:

def max_gain(X, y):
    entropy_root = entropy(y)
    max_gain = 0
    split_feature = ''
    max_gain_dict = []

    for feature in X.columns:
        gini_index = 0
        entropy_list = []
        for c_feature in X[feature].unique():
            split_feature_x = X.iloc[np.where(X[feature]==c_feature)]
            split_y = y[np.where(X[feature]==c_feature)[0]]
            print("E({}={}) = {}".format(feature,c_feature, entropy(split_y)))
            gini_index +=  len(split_feature_x)/len(X)*entropy(split_y)
            entropy_list.append({'class':c_feature, 'entropy':gini_index})

        print("E({}) = {}".format(feature, gini_index))
        info_gain = entropy_root - gini_index
        if(max_gain<info_gain):
            max_gain = info_gain
            split_feature = feature
            max_gain_dict = entropy_list
        print("Info Gain = {}\n".format(info_gain))
    print("Max Gain:{}".format(max_gain))
    return split_feature, max_gain_dict

max_gain(X, y)

E(outlook=overcast) = 0.0
E(outlook=rainy) = 0.9709505944546686
E(outlook=sunny) = 0.9709505944546686
E(outlook) = 0.6935361388961918
Info Gain = 0.2467498197744391

E(temp=hot) = 1.0
E(temp=cool) = 0.8112781244591328
E(temp=mild) = 0.9182958340544896
E(temp) = 0.9110633930116763
Info Gain = 0.029222565658954647

E(humidity=high) = 0.9852281360342515
E(humidity=normal) = 0.5916727785823275
E(humidity) = 0.7884504573082896
Info Gain = 0.15183550136234136

E(windy=False) = 0.8112781244591328
E(windy=True) = 1.0
E(windy) = 0.8921589282623617
Info Gain = 0.04812703040826927

Max Gain:0.2467498197744391


('outlook',
 [{'class': 'overcast', 'entropy': 0.0},
  {'class': 'rainy', 'entropy': 0.3467680694480959},
  {'class': 'sunny', 'entropy': 0.6935361388961918}])

### Next Step

In [9]:
col_name, parent_class_list = max_gain(X, y)
parent_class_list

E(outlook=overcast) = 0.0
E(outlook=rainy) = 0.9709505944546686
E(outlook=sunny) = 0.9709505944546686
E(outlook) = 0.6935361388961918
Info Gain = 0.2467498197744391

E(temp=hot) = 1.0
E(temp=cool) = 0.8112781244591328
E(temp=mild) = 0.9182958340544896
E(temp) = 0.9110633930116763
Info Gain = 0.029222565658954647

E(humidity=high) = 0.9852281360342515
E(humidity=normal) = 0.5916727785823275
E(humidity) = 0.7884504573082896
Info Gain = 0.15183550136234136

E(windy=False) = 0.8112781244591328
E(windy=True) = 1.0
E(windy) = 0.8921589282623617
Info Gain = 0.04812703040826927

Max Gain:0.2467498197744391


[{'class': 'overcast', 'entropy': 0.0},
 {'class': 'rainy', 'entropy': 0.3467680694480959},
 {'class': 'sunny', 'entropy': 0.6935361388961918}]

In [10]:
def get_subtable(X, y, node, class_name):
    X1 = X[X[node]==class_name].reset_index(drop=True)
    y1 = y[np.where(X[node]==class_name)[0]].reset_index(drop=True)
    return X1, y1
get_subtable(X, y, 'outlook', 'rainy')

(  outlook  temp humidity  windy
 0   rainy  mild     high  False
 1   rainy  cool   normal  False
 2   rainy  cool   normal   True
 3   rainy  mild   normal  False
 4   rainy  mild     high   True,
 0    yes
 1    yes
 2     no
 3    yes
 4     no
 Name: play, dtype: object)

In [11]:
for c_feature in parent_class_list:
    class_name = c_feature['class']
    print(class_name)
    X1, y1 = get_subtable(X, y, col_name, class_name)
    max_gain(X1, y1)

overcast
E(outlook=overcast) = 0.0
E(outlook) = 0.0
Info Gain = 0.0

E(temp=hot) = 0.0
E(temp=cool) = 0.0
E(temp=mild) = 0.0
E(temp) = 0.0
Info Gain = 0.0

E(humidity=high) = 0.0
E(humidity=normal) = 0.0
E(humidity) = 0.0
Info Gain = 0.0

E(windy=False) = 0.0
E(windy=True) = 0.0
E(windy) = 0.0
Info Gain = 0.0

Max Gain:0
rainy
E(outlook=rainy) = 0.9709505944546686
E(outlook) = 0.9709505944546686
Info Gain = 0.0

E(temp=mild) = 0.9182958340544896
E(temp=cool) = 1.0
E(temp) = 0.9509775004326937
Info Gain = 0.01997309402197489

E(humidity=high) = 1.0
E(humidity=normal) = 0.9182958340544896
E(humidity) = 0.9509775004326937
Info Gain = 0.01997309402197489

E(windy=False) = 0.0
E(windy=True) = 0.0
E(windy) = 0.0
Info Gain = 0.9709505944546686

Max Gain:0.9709505944546686
sunny
E(outlook=sunny) = 0.9709505944546686
E(outlook) = 0.9709505944546686
Info Gain = 0.0

E(temp=hot) = 0.0
E(temp=mild) = 1.0
E(temp=cool) = 0.0
E(temp) = 0.4
Info Gain = 0.5709505944546686

E(humidity=high) = 0.0
E(humi

In [12]:
def count_classes(label_serie):
    return label_serie.value_counts().to_dict()

def entropy(y):
    """Calculate the Gini Impurity for a list of rows."""
    counts = count_classes(y)
    eps = np.finfo(float).eps
    entropy = 0
    # for each label calculate the entropy
    for lbl in counts:
    
        prob_of_lbl = counts[lbl] / (float(len(y))+eps)
        entropy -= prob_of_lbl * math.log2(prob_of_lbl+eps)
    return entropy

def max_gain(X, y):
    entropy_root = entropy(y)
    max_gain = 0
    split_feature = ''

    for feature in X.columns:
        gini_index = 0
        entropy_list = []
        for c_feature in X[feature].unique():
            split_feature_x = X.iloc[np.where(X[feature]==c_feature)]
            split_y = y[np.where(X[feature]==c_feature)[0]]
            gini_index +=  len(split_feature_x)/len(X)*entropy(split_y)

        info_gain = entropy_root - gini_index
        if(max_gain<info_gain):
            max_gain = info_gain
            split_feature = feature

    return split_feature

def get_subtable(X, y, node, class_name):
    X1 = X[X[node]==class_name].reset_index(drop=True)
    y1 = y[np.where(X[node]==class_name)[0]].reset_index(drop=True)
    return X1, y1

def buildTree(X, y, tree=None): 
    #Get attribute with maximum information gain
    node = max_gain(X, y)
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    c_features = X[node].unique()
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}

    for class_name in c_features:
        X1, y1 = get_subtable(X, y, node, class_name)
        clValue,counts = np.unique(y1,return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][class_name] = clValue[0]                                                    
        else:        
            tree[node][class_name] = buildTree(X1, y1) #Calling the function recursively 
                   
    return tree

buildTree(X, y)

{'outlook': {'overcast': 'yes',
  'rainy': {'windy': {False: 'yes', True: 'no'}},
  'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}

In [14]:
def predict(inst,tree):
    #This function is used to predict for any input variable 
    for nodes in tree.keys():        
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
            
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

tree = buildTree(X, y)
prediction = predict(X.iloc[0], tree)
prediction

'yes'

In [20]:
X.iloc[0]

outlook     overcast
temp             hot
humidity        high
windy          False
Name: 0, dtype: object

In [25]:
y

0     yes
1     yes
2     yes
3     yes
4     yes
5     yes
6      no
7     yes
8      no
9      no
10     no
11     no
12    yes
13    yes
Name: play, dtype: object

In [32]:
predictions = []
for _,row in X.iterrows():
    predictions.append(predict(row, tree))


pd.concat([y, pd.Series(predictions, name='play_pred')],axis=1)

,play,play_pred
0,yes,yes
1,yes,yes
2,yes,yes
3,yes,yes
4,yes,yes
5,yes,yes
6,no,no
7,yes,yes
8,no,no
9,no,no
